In [6]:
# 构建一个非常简单的数据集
sentences = ["我 喜欢 玩具 ", "我 爱 爸爸 ", "我 讨厌 挨打 "]
# 构建词汇表连接所有句子，用空格分隔，然后用set去重，最后转换为list
word_list = list(set("".join(sentences).split()))
# 构建词典
word_to_idx = {word: i for i, word in enumerate(word_list)}
# 构建逆词典
idx_to_word = {i: word for i, word in enumerate(word_list)}
voc_size = len(word_list)  # 词汇表的大小
print("词汇表: ", word_to_idx)
print("词汇表大小：", voc_size)

词汇表:  {'爱': 0, '玩具': 1, '爸爸': 2, '我': 3, '挨打': 4, '喜欢': 5, '讨厌': 6}
词汇表大小： 7


In [7]:
# 构建批处理数据
import torch
import random
batch_size = 2
def make_batch():
    input_batch = []  # 输入数据
    target_batch = []  # 输出数据
    selected_sentences = random.sample(sentences, batch_size)  # 随机选择batch_size个句子
    for sen in selected_sentences: # 遍历每个句子
        word = sen.split()  # 分词
        # 将除最后一个词以外的词作为输入，最后一个词作为输出
        input = [word_to_idx[w] for w in word[:-1]]
        target = word_to_idx[word[-1]]
        input_batch.append(input)  # 添加到输入数据
        target_batch.append(target)  # 添加到输出数据
    input_batch = torch.LongTensor(input_batch)  # 转换为LongTensor
    target_batch = torch.LongTensor(target_batch)  # 转换为LongTensor
    return input_batch, target_batch

input_batch, target_batch = make_batch()  # 生成批处理数据
print("输入批处理数据: ", input_batch)  # 打印输入数据
# 将输入批数据中的每个索引值转化为对应的原始词
input_words = []
for input_idx in input_batch:
    input_words.append([idx_to_word[idx.item()] for idx in input_idx])
print("输入批处理数据对应的原始词: ", input_words)  # 打印输入词
print("目标批处理数据：", target_batch)  # 打印输出数据
# 将输出批数据中的每个索引值转化为对应的原始词
target_words = [idx_to_word[idx.item()] for idx in target_batch]
print("目标批处理数据对应的原始词: ", target_words)  # 打印输出词

输入批处理数据:  tensor([[3, 6],
        [3, 0]])
输入批处理数据对应的原始词:  [['我', '讨厌'], ['我', '爱']]
目标批处理数据： tensor([4, 2])
目标批处理数据对应的原始词:  ['挨打', '爸爸']


In [8]:
n_hidden = 2  # 隐藏层的大小
embedding_size = 2  # 词嵌入的大小

In [9]:
import torch.nn as nn  # 导入神经网络模块
# 定义神经概率语言模型
class NPLM(nn.Module):
    def __init__(self):
        super(NPLM, self).__init__()
        self.C = nn.Embedding(voc_size, embedding_size) # 词嵌入层
        # 用LSTM层替代第一个线性层，其输入大小为embedding_size，隐藏层大小为n_hidden
        self.lstm = nn.LSTM(embedding_size, n_hidden, batch_first=True)
        # 第二个线性层，其输入大小为 n_hidden，输出大小为voc_size，即词汇表的大小
        self.linerar2 = nn.Linear(n_hidden, voc_size)
    def forward(self, X):
        # 输入张量的形状为(batch_size, n_step)
        X = self.C(X)  # X通过词嵌入层，形状变为(batch_size, n_step, embedding_size)
        lstm_out, _ = self.lstm(X)  # X通过LSTM层，形状变为(batch_size, n_step, n_hidden)
        output = self.linerar2(lstm_out[:, -1, :])  # 取最后一个时间步的输出，形状变为(batch_size, voc_size)
        return output

In [10]:
model = NPLM()  # 实例化模型
print("模型结构：", model)  # 打印模型结构

模型结构： NPLM(
  (C): Embedding(7, 2)
  (lstm): LSTM(2, 2, batch_first=True)
  (linerar2): Linear(in_features=2, out_features=7, bias=True)
)


In [11]:
import torch.optim as optim  # 导入优化器
criterion = nn.CrossEntropyLoss()  # 定义损失函数
optimizer = optim.Adam(model.parameters(), lr=0.1)  # 定义优化器
# 训练模型
for epoch in range(5000): # 训练5000次
    optimizer.zero_grad()  # 梯度清零
    input_batch, target_batch = make_batch()  # 生成批处理数据
    output = model(input_batch)  # 模型输出
    loss = criterion(output, target_batch)  # 计算损失
    if (epoch + 1) % 1000 == 0:  # 每1000次打印一次损失
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数

Epoch: 1000 cost = 0.000255
Epoch: 2000 cost = 0.000078
Epoch: 3000 cost = 0.000038
Epoch: 4000 cost = 0.000018
Epoch: 5000 cost = 0.000011


In [12]:
# 进行预测
input_strs = [['我', '喜欢'], ['我', '讨厌']]  # 需要预测的输入序列
# 将输入序列转化为索引序列
input_batch = torch.LongTensor([[word_to_idx[word] for word in input_str] for input_str in input_strs])
# 进行预测
predict = model(input_batch).data.max(1)[1]
# 将预测结果转化为对应的词
predict_words = [idx_to_word[idx.item()] for idx in predict]
for input_seq, pred in zip(input_strs, predict_words):
    print(input_seq, "->", pred)

['我', '喜欢'] -> 玩具
['我', '讨厌'] -> 挨打
